# **KDDCup Data Analytics with PySpark DF: A structured case study**

**`Udemy Course: Hands-on Big Data Practices with PySpark & Spark Tuning`**

**`Author: Amin Karami (PhD, FHEA)`**

##### data source: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [ ]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

In [1]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [2]:
# import SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

In [3]:
# Read and Load Data to Spark
# Data source: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

df = spark.read.text("kddcup.data.gz")
df.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
# Print data
df.show()

+--------------------+
|               value|
+--------------------+
|0,tcp,http,SF,215...|
|0,tcp,http,SF,162...|
|0,tcp,http,SF,236...|
|0,tcp,http,SF,233...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,238...|
|0,tcp,http,SF,235...|
|0,tcp,http,SF,234...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,181...|
|0,tcp,http,SF,184...|
|0,tcp,http,SF,185...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,181...|
|0,tcp,http,SF,236...|
|0,tcp,http,SF,233...|
|0,tcp,http,SF,238...|
|0,tcp,http,SF,235...|
|0,tcp,http,SF,234...|
|0,tcp,http,SF,239...|
+--------------------+
only showing top 20 rows



In [5]:
# Split data (if needed)

from pyspark.sql.functions import split

split_col = split(df['value'], ',')
df = df.withColumn('Protocol', split_col.getItem(1)) \
       .withColumn('Service', split_col.getItem(2)) \
       .withColumn('flag', split_col.getItem(3)) \
       .withColumn('src_bytes', split_col.getItem(4)) \
       .withColumn('dst_bytes', split_col.getItem(5)) \
       .withColumn('urgent', split_col.getItem(8)) \
       .withColumn('num_failed_logins', split_col.getItem(10)) \
       .withColumn('root_shell', split_col.getItem(13)) \
       .withColumn('guest_login', split_col.getItem(21)) \
       .withColumn('label', split_col.getItem(41)) \
       .drop('value')

df.show()

+--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
|Protocol|Service|flag|src_bytes|dst_bytes|urgent|num_failed_logins|root_shell|guest_login|  label|
+--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
|     tcp|   http|  SF|      215|    45076|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      162|     4528|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      236|     1228|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      233|     2032|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      239|      486|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      238|     1282|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      235|     1337|     0|                0|         0|          0|normal.|


In [6]:
# Increase the number of partitions (if needed) and Build a Temp table

df = df.repartition(10)
print(df.rdd.getNumPartitions())

df.createOrReplaceTempView("df_KDDCup")

10


## Question 1: Count the number of connections for each label

In [10]:
spark.conf.set('spark.sql.adaptive.enabled', True)
spark.conf.set('spark.sql.adaptive.coalescePartitions.enabled', True)

In [11]:
df1 = df.groupBy('label').count()
df1.rdd.getNumPartitions()

5

In [12]:
df2 = df.groupBy('label').count().orderBy('count', ascending= False)
df2.rdd.getNumPartitions()

5

## Question 2:  Get the list of `Protocols`that are `normal` and `vulnerable to attacks`, where there is NOT `guest login` to the destination addresses.


In [13]:
sql_query = """ SELECT Protocol,
                CASE label
                  WHEN 'normal.' THEN 'no attack'
                  ELSE 'attack'
                END AS State,
                COUNT(*) as freq
              FROM df_KDDCup
              WHERE guest_login != 1
              GROUP BY Protocol, State
              ORDER BY Protocol DESC
            """

df3 = spark.sql(sql_query)
df3.rdd.getNumPartitions()

4

## Question 3: Apply Some Descriptive Statistics on Numerical Data

In [ ]:
# PySpark provides built-in standard Aggregate functions defines in DataFrame API
from pyspark.sql.functions import *

summary = df.select(mean(df.src_bytes).alias('AVG(src_byte)'),
                    stddev(df.src_bytes).alias('STD(src_byte)'),
                    min(df.src_bytes).alias('MIN(src_byte)'),
                    max(df.src_bytes).alias('MAX(src_byte)'),
                    last(df.src_bytes).alias('LAST(src_byte)'),
                    skewness(df.src_bytes).alias('SKEW(src_byte)'),
                  )
summary.show()

In [ ]:
groups = df.groupBy("Protocol")
groups.agg({'src_bytes':'mean', 'dst_bytes':'stddev'}).show()

## Question 4: A descriptive stats based on `Protocols` and `Labels`


In [ ]:
sql_query = """
                           SELECT protocol,
                             CASE label
                               WHEN 'normal.' THEN 'no attack'
                               ELSE 'attack'
                             END AS state,
                             COUNT(*) as total_freq,
                            ROUND(AVG(src_bytes), 2) as mean_src_bytes,
                            ROUND(AVG(dst_bytes), 2) as mean_dst_bytes,
                            SUM(urgent) as sum_urgent,
                            SUM(num_failed_logins) as sum_failed_logins,
                            SUM(root_shell) as sum_root_shell,
                            SUM(guest_login) as sum_guest_login
                           FROM df_KDDCup
                           GROUP BY protocol, state
                           ORDER BY 3 DESC
                           """
spark.sql(sql_query).show()

## Question 5: Get the frquency of `services` for the original `UDP and ICMP` based `attacks`
(hint 1: original attacks: `[dos, u2r, r2l, and probe]`)

(hint 2: returns the `services` and `protocols` center justified)

In [ ]:
from pyspark.sql.types import StringType

def Attack_Category(item):
  if item.replace(".", "") in ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']:
    return "DoS"
  elif item.replace(".", "") in ['buffer_overflow', 'loadmodule', 'perl', 'rootkit']:
    return "U2R"
  elif item.replace(".", "") in ['ftp_write', 'guess_passwd', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']:
    return "R2L"
  else: return "Probe" 

def Center_Justify(item):
  return item.center(10)


spark.udf.register("OrginalAttacks", Attack_Category, StringType())
spark.udf.register("TextJustify", Center_Justify, StringType())

sql_query = """
                SELECT 
                  TextJustify(service) as service,
                  TextJustify(protocol) as protocol,
                  OrginalAttacks(label) as new_label,
                  COUNT(*) as freq
                FROM df_KDDCup
                WHERE (protocol = 'udp' OR protocol = 'icmp') AND label != 'normal.'
                GROUP BY service, new_label, protocol
                ORDER BY freq DESC
          """

spark.sql(sql_query).show()